# Model pipeline draft
inspired by https://www.tensorflow.org/tutorials/images/transfer_learning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import accuracy_score
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models, Sequential, Input, Model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.data.experimental import cardinality
from tensorflow.data.experimental import AUTOTUNE
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
path0 = r'C:\Users\pitip\code\ClaireLeroux44\ArtRecognition'
n_artist = 3
n_embedding = 100

Steps:
- load images from dataset (after folder prep) : image from dataset
- load VGG16
- data augmentation
- change VGG16 last layer (add N artist id)
- train VGG16 on dataset
- predict artist on test set + accuracy

## Load images from directory

Load images + check classes + create train/val/test dataset

In [3]:
train_dir = os.path.join(path0, 'raw_data', 'test_VGG16', 'Train')
test_dir = os.path.join(path0, 'raw_data', 'test_VGG16', 'Test')
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(train_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE, label_mode='categorical')
test_dataset = image_dataset_from_directory(test_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE, label_mode='categorical')

Found 15 files belonging to 3 classes.
Found 9 files belonging to 3 classes.


In [4]:
class_names = train_dataset.class_names
class_names

['class_1', 'class_2', 'class_3']

In [5]:
train_batches = cardinality(train_dataset)
validation_dataset = train_dataset.take(train_batches // 5)
train_train_dataset = train_dataset.skip(train_batches // 5)

In [6]:
train_train_dataset = train_train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [7]:
print('Number of validation batches: %d' % cardinality(validation_dataset))
print('Number of test batches: %d' % cardinality(test_dataset))

Number of validation batches: 0
Number of test batches: 1


In [8]:
for image_batch, labels_batch in test_dataset:
    print(image_batch.shape)
    print(labels_batch.shape)

(9, 224, 224, 3)
(9, 3)


## Create VGG16 root model

VGG16 + flatten layer + dense layer of ndense_1 neurons

In [9]:
def VGG16_model(ndense_1=100):
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_model.trainable = False

    model_layers = [base_model, layers.Flatten(), 
                  layers.Dense(ndense_1, activation='relu')]
    model = models.Sequential(layers = model_layers)
    return model

model1 = VGG16_model(ndense_1=n_embedding)
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               2508900   
Total params: 17,223,588
Trainable params: 2,508,900
Non-trainable params: 14,714,688
_________________________________________________________________


## Data augmentation (included in the model)
https://www.tensorflow.org/tutorials/images/data_augmentation

In [10]:
data_augmentation = Sequential([layers.experimental.preprocessing.RandomFlip('horizontal'),
                                layers.experimental.preprocessing.RandomRotation(0.1),
                                layers.experimental.preprocessing.RandomContrast(0.1),
                                layers.experimental.preprocessing.RandomZoom(height_factor=0.2),
                                layers.experimental.preprocessing.RandomTranslation(height_factor=0.2, width_factor=0.2)
])

## Complete model generation

Data_aug + preprocess (VGG16) + model_VGG16 + prediction_layer

In [11]:
inputs = Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = model1(x)
prediction_layer = layers.Dense(n_artist, activation='softmax')
outputs = prediction_layer(x)
model = Model(inputs, outputs)

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [13]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 224, 224, 3)       0         
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_BiasAdd (TensorF [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 100)               17223588  
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 17,223,891
Trainable params: 2,509,203
Non-trainable params: 14,714,688
__________________________________

## Fit du model

In [14]:
history = model.fit(train_train_dataset, epochs=10, validation_data=test_dataset, batch_size=32, 
                    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 9.2051 - accuracy: 0.3333 - val_loss: 46.1709 - val_accuracy: 0.3333
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 32.8460 - accuracy: 0.4000 - val_loss: 45.2194 - val_accuracy: 0.2222
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 29.9141 - accuracy: 0.4000 - val_loss: 35.9045 - val_accuracy: 0.4444
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 15.1263 - accuracy: 0.4667 - val_loss: 36.6399 - val_accuracy: 0.3333
Epoch 5/10
1/1 [==============================] - 3s 3s/step - loss: 9.5306 - accuracy: 0.6667 - val_loss: 24.8580 - val_accuracy: 0.3333
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 4.9791 - accuracy: 0.8000 - val_loss: 22.3400 - val_accuracy: 0.2222
Epoch 7/10
1/1 [==============================] - 2s 2s/step - loss: 5.4240 - accuracy: 0.6000 - val_loss: 21.5580 - val_accuracy: 0.2222
Epoch 8/10
1/1 [===============

## Plot history and compute accuracy

In [15]:
fig, ax = plt.subplots(1, 2, figsize=(15, 4))
ax[0].plot(history.history['loss'], label='Train set loss')
ax[0].plot(history.history['val_loss'], label='Validation set loss')
ax[0].legend()
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')
ax[1].plot(history.history['accuracy'], label='Train set accuracy')
ax[1].plot(history.history['val_accuracy'], label='Validation set accuracy')
ax[1].legend()
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Accuracy');

In [16]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

1/1 [==============================] - 0s 3ms/step - loss: 13.9697 - accuracy: 0.3333
Test accuracy : 0.3333333432674408
